## Load the dataset

In [1]:
import pandas as pd 

df = pd.read_csv('./corpus/TweetSentiment.csv')
df = df[['cleaned_text', 'label']]
df.dropna(inplace=True)
df.sample(5)

,cleaned_text,label
3582,the wedding over everyone has gone home newlyw...,0.0
30114,cinema tonight half the price monday,2.0
28871,have both her cds and know them both heart rea...,0.0
14648,rather sit bench with friendly psychiatric pat...,2.0
7737,try direct message here twitter,0.0


## Split the dataset into train and test sets

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_text'].tolist(),
    df['label'].tolist(),
    test_size = 0.2,
    stratify = df['label'].tolist(),
    random_state = 64
)

In [3]:
from collections import Counter

# print the statistics of train and test sets
print(f'Train data instances: {len(X_train)}\nClass distribution: {Counter(y_train)}')
print(f'\nTest data instances: {len(X_test)}\nClass distribution: {Counter(y_test)}')

Train data instances: 24144
Class distribution: Counter({0.0: 9660, 2.0: 7560, 1.0: 6924})

Test data instances: 6037
Class distribution: Counter({0.0: 2416, 2.0: 1890, 1.0: 1731})


In [4]:
import random

# create iterator: list of tuples -> (label, text)
train_data = list(zip(y_train, X_train))
test_data = list(zip(y_test, X_test))

# display training samples
random.choices(train_data, k = 5)

[(0.0, 'wasnt was with the the time and now omw class what this new kaggra'),
 (2.0, 'yeah checked pretty nice site'),
 (2.0, 'omg that toooo funny'),
 (0.0, 'company policy has been for the last two places ive worked'),
 (1.0,
  'sometimes forget favorite stars are real people too made orange chicken last night and cut his finger off sad')]

## DataLoader

In [5]:
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# from torchtext.data.utils import get_tokenizer
# tokenizer = get_tokenizer('basic_english')

def tokenizer(x):
    return x.lower().split()

def yield_tokens(data_iterator):
    for _, text in data_iterator:
        yield tokenizer(text)

In [7]:
from torchtext.vocab import build_vocab_from_iterator

# build vocabulary
VOCAB = build_vocab_from_iterator(yield_tokens(train_data), specials=[''])
VOCAB.set_default_index(VOCAB[''])

In [8]:
# create pipelines
TEXT_PIPELINE = lambda x: VOCAB(tokenizer(x))
LABEL_PIPELINE = lambda x: int(x)

# pipelines in action
print(TEXT_PIPELINE('This is an example'))
print(LABEL_PIPELINE('2'))

[16, 0, 0, 7032]
2


In [9]:
# batch collate function
def collate_batch(batch):
    labels, texts, offsets = [], [], [0]
    for (label, text) in batch:
        labels.append(LABEL_PIPELINE(label))
        _texts = torch.tensor(TEXT_PIPELINE(text), dtype=torch.int64)
        texts.append(_texts)
        offsets.append(_texts.size(0))
    labels = torch.tensor(labels, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    texts = torch.cat(texts)
    return labels.to(DEVICE), texts.to(DEVICE), offsets.to(DEVICE)

In [10]:
from torch.utils.data import DataLoader

# hyperparameters
EPOCHS = 25
LEARNING_RATE = 0.5
BATCH_SIZE = 64

# dataloaders
train_loader = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle = True, collate_fn = collate_batch)  # train data is train iterator
test_loader = DataLoader(test_data, batch_size = BATCH_SIZE, shuffle = True, collate_fn = collate_batch)  # test data is test iterator

## Text Classification Model
A feed-forward neural network

In [11]:
from torch import nn
import torch.nn.functional as F

class FeedForwardNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(FeedForwardNN, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.68
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()
        self.fc4.weight.data.uniform_(-initrange, initrange)
        self.fc4.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = F.relu(self.fc1(embedded))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [12]:
NUM_CLASSES = len(set([label for (label, text) in train_data]))
VOCAB_SIZE = len(VOCAB)
EMBED_SIZE = 128

# initialize the model
model = FeedForwardNN(VOCAB_SIZE, EMBED_SIZE, NUM_CLASSES).to(DEVICE)

In [13]:
# loss fn, optimizer, scheduler
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

## Train and Evaluate the Model

In [14]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 100
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

In [15]:
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [16]:
total_accu = None

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_loader)
    accu_val = evaluate(test_loader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'test accuracy {:8.3f} '.format(epoch, time.time() - epoch_start_time, accu_val))
    print('-' * 59)

| epoch   1 |   100/  378 batches | accuracy    0.366
| epoch   1 |   200/  378 batches | accuracy    0.405
| epoch   1 |   300/  378 batches | accuracy    0.417
-----------------------------------------------------------
| end of epoch   1 | time:  3.61s | test accuracy    0.427 
-----------------------------------------------------------
| epoch   2 |   100/  378 batches | accuracy    0.445
| epoch   2 |   200/  378 batches | accuracy    0.443
| epoch   2 |   300/  378 batches | accuracy    0.445
-----------------------------------------------------------
| end of epoch   2 | time:  4.20s | test accuracy    0.433 
-----------------------------------------------------------
| epoch   3 |   100/  378 batches | accuracy    0.466
| epoch   3 |   200/  378 batches | accuracy    0.464
| epoch   3 |   300/  378 batches | accuracy    0.474
-----------------------------------------------------------
| end of epoch   3 | time:  4.84s | test accuracy    0.463 
----------------------------------

## Test the Model on Input Text

In [17]:
sentiment_label = {2:"Positive", 1: "Negative", 0: "Neutral"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() 

In [18]:
# inp_text = "Soooooo wish I could, but im in school and myspace is completely blocked"
# inp_text = "The product is not good"
inp_text = "It's super fun"

print(f"This is a {sentiment_label[predict(inp_text, TEXT_PIPELINE)]} tweet")

This is a Positive tweet


## References

In [19]:
# https://github.com/rsreetech/PyTorchTextClassificationCustomDataset/blob/main/PyTorchTweetTextClassification.ipynb